In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import datetime, time
from datetime import datetime

import numpy as np
import math

import os
import pymongo


from functions.Plots import *
from functions.PowerCalculation import *
from functions.Resample import *
from functions.StopPointDetection import *
from functions.Elevation import getElevation, getAngles
from functions.ImportVehicle import *

data_dir = 'D:\\Telematics\\Car Track\\Data\\Joined'

cars = os.listdir(data_dir)

client = pymongo.MongoClient()

db = client['fleet']
processed= db['Clean']
Trips = db['Trips']

In [ ]:
pyspark.__version__


In [ ]:
def getDayGroups(n):
    
    
    a = n//3
    b = n%3
    weeks = []
    
    for i in range(a):
        days = []
        
        for j in range(3):
            
            x = 3*i
            
            days.append(x+j)
        weeks.append(days)
    temp = []
    
    for z in range(b):
        temp.append(x+j+1+z)
    weeks.append(temp)
        
    return weeks
    

In [2]:
spark = SparkSession.builder.appName("App").getOrCreate()

In [ ]:
file = os.path.join(data_dir, cars[2])

df = spark.read.csv(file, inferSchema=True, header=True)

vehicleTag = cars[2].split(".csv")[0]

print(vehicleTag)

df_clean = cleanDF(df)

car_days = getDaysDF(df_clean, getDays(df_clean))

#weeks = getDayGroups(len(car_days))
#print(weeks)

In [3]:
for car in cars:
    
    file = os.path.join(data_dir, car)

    df = spark.read.csv(file, inferSchema=True, header=True)

    vehicleTag = car.split(".csv")[0]

    print(vehicleTag)

    df_clean = cleanDF(df)

    car_days = getDaysDF(df_clean, getDays(df_clean))

    for i in range(len(car_days)):


        vehicle = Vehicle(vehicleTag, car_days[i])

        vehicle.resample()

        #plotTimeSeries(vehicle.getDate(), vehicle.getParameter("Longitude"))

        time, velocity, lat, long = vehicle.getRegularTimestep()
        print(i)

        print("Resample Done")

        SP = zeroVP(time, velocity, lat, long)

        acc = CD(velocity, 10)

        #elevation = getElevation(lat, long)

        totalDist, s, c = getTotalDistance(lat, long)

        #angles = getAngles(elevation, s)

        _, power = getPower(1300, acc, velocity, 0.02, 0.33, 1.75*1.48, np.zeros((len(time))))

        energy = getTotalEnergy(time, power, 10)

        epk = EPK(energy, totalDist)

        #powerExport = ExportPower(vehicleTag, time, velocity, lat, long, acc, power, np.zeros((len(time))),np.zeros((len(time))), s, c)

        #powerExport.exportMongoDB(processed)

        print()
        print(f'Energy: {energy}, Total Distance: {totalDist}, EPK: {epk}')
        print()

        trips = getTrips(SP, time, power, lat, long)

        tripExport = ExportTrip(vehicleTag, trips)

        tripExport.exportMongoDB(Trips)

        print("\n")



KF44LNGP
0
Resample Done

Energy: 81.00010848979593, Total Distance: 423.28639236391695, EPK: 0.19136005775530995



1
Resample Done

Energy: 62.52024131201979, Total Distance: 341.18459415999513, EPK: 0.18324462001558472



2
Resample Done

Energy: 93.11779563876716, Total Distance: 500.9964252313825, EPK: 0.18586518974813285



3
Resample Done

Energy: 76.17439881831264, Total Distance: 450.01111607610187, EPK: 0.16927226038885382



4
Resample Done

Energy: 49.45040320165217, Total Distance: 277.10303587946566, EPK: 0.1784549311944822



5
Resample Done

Energy: 95.42978024510121, Total Distance: 490.9694868454078, EPK: 0.19437008368536213



6
Resample Done

Energy: 59.32774221337929, Total Distance: 380.4134378889728, EPK: 0.15595595818750926



7
Resample Done

Energy: 57.777589976131836, Total Distance: 336.96145168533354, EPK: 0.17146646801037224



8
Resample Done

Energy: 29.621483647756754, Total Distance: 183.0606580430624, EPK: 0.16181239576222176



9
Resample Done

Energ

10
Resample Done

Energy: 62.43552173498876, Total Distance: 386.05450963383703, EPK: 0.16172721772945298



11
Resample Done

Energy: 70.01187213313554, Total Distance: 384.9803512254536, EPK: 0.18185829980744894



12
Resample Done

Energy: 20.558244020803375, Total Distance: 123.24108559819896, EPK: 0.16681323376060728



13
Resample Done

Energy: 79.2017569035969, Total Distance: 449.7733413124315, EPK: 0.17609259960247406



14
Resample Done

Energy: 56.98325338137442, Total Distance: 265.42907557391555, EPK: 0.21468353931521705



15
Resample Done

Energy: 67.3448057523746, Total Distance: 347.59064894127596, EPK: 0.19374746115150018



16
Resample Done

Energy: 89.44501323843369, Total Distance: 495.5641035572024, EPK: 0.18049130797890642



17
Resample Done

Energy: 160.3407530390455, Total Distance: 799.3830131341745, EPK: 0.20058063582110758



18
Resample Done

Energy: 66.51272138498868, Total Distance: 421.01192490774537, EPK: 0.15798298682291087



19
Resample Done

Energy

In [ ]:
plotTimeSeries(getDateTime(time), long)

In [ ]:
plotTimeSeries(getDateTime(time), elevation)
plotTimeSeries(getDateTime(time), angles)

In [ ]:
plotTimeSeries(getDateTime(time), velocity)
plotTimeSeries(getDateTime(time), acc)
plotTimeSeries(getDateTime(time), power)

In [ ]:
plotTimeSeries(getDateTime(time), velocity)
plotTimeSeries(getDateTime(time), acc)
plotTimeSeries(getDateTime(time), power)

In [ ]:
total , s, c = getTotalDistance(lat, long)
print(total)


In [ ]:
plotTimeSeries(time, s)
plotTimeSeries(time, c)